In [ ]:
import nltk
import sacrebleu
from nltk.translate.meteor_score import meteor_score
from bert_score import score
import pandas as pd

In [2]:
model_name = "legal_t5"
output_csv_file = f"{model_name}_translation_results.csv"
results = pd.read_csv(output_csv_file)
results.head(2)

,input,generated,reference
0,Tel: +31 (0)182 557 755,Tel: +31 (0)182 557 755,Tel: +31 (0)182 557 755
1,5.3 Données de sécurité précliniques Les donné...,5.3 Pre-clinical safety data Preclinial data f...,5.3 Preclinical safety data Preclinical data r...


In [3]:
# BLEU score
results["generated"] = results["generated"].astype(str)
results["reference"] = results["reference"].astype(str)

# Extract the generated and reference translations
generated = results["generated"].tolist()
references = [results["reference"].tolist()]

# Calculate BLEU score
bleu = sacrebleu.corpus_bleu(generated, references)

print(f"BLEU score: {bleu.score}")

BLEU score: 34.11146183808727


In [4]:
# METEOR score
nltk.download('wordnet')

results["generated"] = results["generated"].astype(str)
results["reference"] = results["reference"].astype(str)

# Example DataFrame
data = {
    "hypothesis": results["generated"],
    "reference": results["reference"]
}
combined_results = pd.DataFrame(data)

# Tokenize the strings
combined_results["hypothesis"] = combined_results["hypothesis"].apply(lambda x: x.split())
combined_results["reference"] = combined_results["reference"].apply(lambda x: x.split())

# Calculate METEOR scores for each row
combined_results["meteor_score"] = combined_results.apply(lambda row: meteor_score([row["reference"]], row["hypothesis"]), axis=1)

overall_meteor_score = combined_results["meteor_score"].mean()

print(f"Overall METEOR score: {overall_meteor_score}")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\micha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Overall METEOR score: 0.521641947816436


In [ ]:
# BERT Score
data = {
    "hypothesis": results["generated"],
    "reference": results["reference"]
}
combined_results = pd.DataFrame(data)

# Extract the hypothesis and reference as lists
hypotheses = combined_results["hypothesis"].tolist()
references = combined_results["reference"].tolist()

# Set a batch size for processing
batch_size = 128

# Initialize lists to store batch results
all_F1_scores = []

# Process the data in batches
for i in range(0, len(hypotheses), batch_size):
    # Get the current batch
    batch_hypotheses = hypotheses[i:i+batch_size]
    batch_references = references[i:i+batch_size]
    
    # Calculate BERTScore for the current batch
    P, R, F1 = score(batch_hypotheses, batch_references, model_type="distilbert-base-uncased", lang="en", verbose=True)
    
    # Append the F1 scores to the list
    all_F1_scores.extend(F1.tolist())

    clear_output(wait=True)

# Compute the overall BERTScore (mean of all F1 scores)
overall_bert_score = sum(all_F1_scores) / len(all_F1_scores)
print(f"Overall BERTScore: {overall_bert_score}")


Overall BERTScore: 0.9162254946862581
